<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2022/blob/main/notebooks/5-model-rivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wstęp

W tym skrypcie zastosujemy metodę opartą na pracy [Rivers, D. (2007). Sampling for web surveys, Proceedings of the Survey Research Methods
Section of the American Statistical Association, pp. 1–26.](https://static.texastribune.org/media/documents/Rivers_matching4.pdf). 

W tym ćwiczeniu porównamy dwa podejścia:

1. wykorzystując zmienne $\mathbf{X}$ do łączenia
2. wykorzystując wartości przewidywane $\hat{y}_i$ na podstawie modelu regresji logistycznej

Poszukiwania sąsiadów dokonamy wykorzystując metodę K-D tree dla przybliżonych sąsiadów oprogramowaną w pakiecie `RANN` (funkcja `nn2`), która opiera się na bibliotece [ANN w C++](http://www.cs.umd.edu/~mount/ANN/).

# Pakiety i dane

Instalujemy pakiet

In [1]:
install.packages("RANN")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Ładujemy pakiety

In [ ]:
library(tidyverse) 
library(RANN) 

Pobieramy dane z zajęć

In [3]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")

In [4]:
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE) %>%
    mutate(woj = as.factor(woj),
           zawod_kod2 = as.factor(zawod_kod2))
tail(dane)

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<fct>,<fct>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
20941,NA,a6a20c0f40c36af79446a53cb1af98dac84ca2cf,NA,2,S,G,8,8,NA,1,TRUE,TRUE,2,1
20942,NA,a6a20c0f40c36af79446a53cb1af98dac84ca2cf,NA,2,S,G,8,9,NA,1,TRUE,TRUE,2,1
20943,NA,5ab1d632da4eda181d7a454f71dd69e4433b6aa5,NA,2,D,H,8,9,NA,1,FALSE,TRUE,1,0
20944,NA,a108d5030e205dc1632b5b8b3eddb725821adfa5,NA,2,S,I,8,7,NA,1,TRUE,TRUE,1,0
20945,NA,a108d5030e205dc1632b5b8b3eddb725821adfa5,NA,2,S,I,8,9,NA,1,TRUE,TRUE,15,0
20946,NA,d5c8a81acfb94539956a0d87c37a233e21ec435f,NA,2,M,H,8,7,NA,1,FALSE,TRUE,1,0


## Metoda Rivers'a -- łączenie na podstawie zmiennych X

Na tę chwilę założymy, że zbiory są rozłączne!

In [5]:
proba_nielosowa <- dane %>% filter(is.na(id_popyt))
proba_losowa <- dane %>% filter(!is.na(id_popyt))

Tworzymy macierz X_A (próba nielosowa) i X_B (próba losowa)

In [6]:
X_A <- model.matrix(~ -1 + sek + klasa_pr + sekc_pkd + woj + zawod_kod2, data = proba_nielosowa)
X_B <- model.matrix(~ -1 + sek + klasa_pr + sekc_pkd + woj + zawod_kod2, data = proba_losowa)
head(X_B)

,sek,klasa_prD,klasa_prM,klasa_prS,sekc_pkdD.E,sekc_pkdF,sekc_pkdG,sekc_pkdH,sekc_pkdI,sekc_pkdJ,⋯,woj28,woj30,woj32,zawod_kod22,zawod_kod23,zawod_kod24,zawod_kod25,zawod_kod27,zawod_kod28,zawod_kod29
1,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
6,1,1,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0


In [7]:
dim(X_A)
dim(X_B)

[1] 10972    39

[1] 9974   39

W zbiorze A mamy 10972 obserwacje, a zbiorze B 9974. Oznacza to, że musimy znaleźć 9974 jednostek ze zbioru A dla jednostek ze zbioru B (mogą się powtarzać).

In [8]:
sasiedzi <- nn2(data = X_A, query = X_B, k = 1)
str(sasiedzi,1)

List of 2
 $ nn.idx  : int [1:9974, 1] 9509 1293 603 3148 3148 8203 7651 8203 6687 10723 ...
 $ nn.dists: num [1:9974, 1] 1 0 0 0 0 ...


Funkcja `nn2` zwraca listę składającą się z dwóch wektorów:

+ `nn.indx` -- indeksy (numery wierszy) ze zbioru `X_A`,
+ `nn.dists` -- odległości między sąsiadami ze zbiorów `X_A` i `X_B` 

Wyznaczamy teraz estymator $\hat{\theta}_{M1}$ czyli chcemy poznać odsetek podmiotów poszukujących pracowników na jedną zmianę.

In [9]:
indeksy <- sasiedzi$nn.idx[,1]
y <- proba_nielosowa$jedna_zmiana[indeksy]

naiwny <- weighted.mean(proba_nielosowa$jedna_zmiana, proba_nielosowa$wolne_miejsca_cbop)

theta_1 <- weighted.mean(y, proba_losowa$waga*proba_losowa$wolne_miejsca)

print(c(naiwny = naiwny, theta_1 = theta_1))

   naiwny   theta_1 
0.5204615 0.7216355 


Oznacza to, że około 72% podmiotów poszukuje pracowników do pracy na jedną zmianę. Gdybyśmy wykorzystali wyłącznie dane z CBOP ten odstek wynosiłby 52%. Porówjamy teraz wyniki z wykorzystaniem łączenia na podstawie $\hat{y}$.

## Metoda Rivers'a -- łączenie na podstawie $\hat{y}$

Budujemy model

In [10]:
model_a <- glm(formula = jedna_zmiana ~ sek + klasa_pr + sekc_pkd + woj + zawod_kod2, 
               data = proba_nielosowa,
               family = binomial(),
               weights = proba_nielosowa$wolne_miejsca_cbop)

y_hat_a <- fitted(model_a)
y_hat_b <- predict(model_a, proba_losowa, type = "response")

summary(y_hat_b)
summary(y_hat_a)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.04645 0.27988 0.49770 0.53685 0.81273 0.99871 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.04822 0.43034 0.65296 0.62312 0.85215 0.99835 

Szukamy sąsiadów na podstawie wartości przewidywanych

In [11]:
sasiedzi_hat <- nn2(data = y_hat_a, query = y_hat_b, k = 1)
str(sasiedzi_hat,1)

List of 2
 $ nn.idx  : int [1:9974, 1] 453 1293 603 3148 3148 2279 9002 7070 2592 8112 ...
 $ nn.dists: num [1:9974, 1] 5.51e-05 0.00 0.00 0.00 0.00 ...


Estymator

In [12]:
indeksy_hat <- sasiedzi_hat$nn.idx[,1]
y_hat <- proba_nielosowa$jedna_zmiana[indeksy_hat]
theta_2 <- weighted.mean(y_hat, proba_losowa$waga*proba_losowa$wolne_miejsca)
print(c(naiwny = naiwny, theta_1 = theta_1, theta_2 = theta_2))

   naiwny   theta_1   theta_2 
0.5204615 0.7216355 0.7141090 


Wykorzystując podejście oparte na wartościach przewidywanych ($\hat{y}$) otrzymujemy oszacowanie niższe tj. 71% ale mając na względzie obciążenie $O_p (n^{1/2-1/p})$ bardziej ufamy wynikowi na podstawie wartości przewidywanych.